### Importação das Libs

In [1]:
#!pip install import_ipynb
#import import_ipynb
#import Fase2_PreProcessamento.ipynb

In [2]:
import pandas as pd
import numpy as np
import pickle

#Pacotes Estatísticos
import scipy

#ML
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#Gráficos
import seaborn as sns
from matplotlib import pyplot as plt

### Variáveis de Configuração

In [3]:
local_origem = "C:/Lab Transp/Dados Processados/"
local_destino = "C:/Lab Transp/Dados Processados/"


#Configurações Gráficas
plt.rcParams['figure.figsize'] = [20, 8]
sns.set_style("darkgrid")

#Determinação das Datas de Corte
data_inicio ='2012-01-01'  #Data de início para a análise
data_fim ='2023-12-31'     #Data de fim para a análise

data_corte_pre_pandemia = '2018-01-01'     #Data Comparativa de Corte Inicial para o Período Pré-Pandemia
data_corte_pos_pandemia = '2022-01-01'     #Data Comparativa de Corte Final para o Período Pré-Pandemia

fonte_1 = {'family': 'sans serif', 'size' : 20}
fonte_2 = {'family': 'sans serif', 'size' : 15}

In [4]:
df_final_tratado = pd.read_csv(local_destino + "\\" + 'df_final_tratado' + ".csv", encoding="latin-1", decimal=",")
df_final_tratado.info()

<ipython-input-4-2a78f8349078>:1: DtypeWarning: Columns (2,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final_tratado = pd.read_csv(local_destino + "\\" + 'df_final_tratado' + ".csv", encoding="latin-1", decimal=",")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3438312 entries, 0 to 3438311
Data columns (total 26 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   Nome_Arquivo                   object 
 1   Data                           object 
 2   Area                           object 
 3   Empresa                        object 
 4   Linha_Completa                 object 
 5   Tipo                           object 
 6   Total                          float64
 7   Linha                          object 
 8   Nome_Ponto_Inicial             object 
 9   Sub_Prefeitura                 object 
 10  Lat_Long_Ponto_Inicial         object 
 11  Nome_Ponto_Final               object 
 12  Lat_Long_Ponto_Final           object 
 13  Ano                            int64  
 14  Mes                            int64  
 15  Dia                            int64  
 16  Volume_Passageiros_Pagantes    float64
 17  Volume_Pagantes_Dinheiro       float64
 18  Vo

In [5]:
#Conversão de e tratamento de campos String:
df_final_tratado["Nome_Arquivo"] = df_final_tratado["Nome_Arquivo"].astype(str)
df_final_tratado["Area"] = df_final_tratado["Area"].astype(str)
df_final_tratado["Empresa"] = df_final_tratado["Empresa"].astype(str)
df_final_tratado["Linha_Completa"] = df_final_tratado["Linha_Completa"].astype(str)
df_final_tratado["Tipo"] = df_final_tratado["Tipo"].astype(str)
df_final_tratado["Linha"] = df_final_tratado["Linha"].astype(str)
df_final_tratado["Nome_Ponto_Inicial"] = df_final_tratado["Nome_Ponto_Inicial"].astype(str)
df_final_tratado["Sub_Prefeitura"] = df_final_tratado["Sub_Prefeitura"].astype(str)
df_final_tratado["Lat_Long_Ponto_Inicial"] = df_final_tratado["Lat_Long_Ponto_Inicial"].astype(str)
df_final_tratado["Nome_Ponto_Final"] = df_final_tratado["Nome_Ponto_Final"].astype(str)
df_final_tratado["Lat_Long_Ponto_Final"] = df_final_tratado["Lat_Long_Ponto_Final"].astype(str)

#Conversão e tratamento de campo data
df_final_tratado["Data"] = pd.to_datetime(df_final_tratado["Data"].astype(str))

#Conversão e tratamento de campos númericos
df_final_tratado["Volume_Passageiros_Gratuidade"] = df_final_tratado["Volume_Passageiros_Gratuidade"].astype(float)
df_final_tratado["Volume_BU_Comum"] = df_final_tratado["Volume_BU_Comum"].astype(float)
df_final_tratado["Volume_BU_Vale_Transporte"] = df_final_tratado["Volume_BU_Vale_Transporte"].astype(float)
df_final_tratado["Volume_Integracao"] = df_final_tratado["Volume_Integracao"].astype(float)
df_final_tratado["Volume_BU_Estudante"] = df_final_tratado["Volume_BU_Estudante"].astype(float)
df_final_tratado["Volume_Passageiros_BU_VT"] = df_final_tratado["Volume_Passageiros_BU_VT"].astype(float)
df_final_tratado["Volume_Passageiros_BU_VT_DIN"] = df_final_tratado["Volume_Passageiros_BU_VT_DIN"].astype(float)

df_final_tratado["Ano"] = df_final_tratado["Ano"].fillna(df_final_tratado["Data"].dt.year).astype(int)
df_final_tratado["Mes"] = df_final_tratado["Mes"].fillna(df_final_tratado["Data"].dt.month).astype(int)
df_final_tratado["Dia"] = df_final_tratado["Dia"].fillna(df_final_tratado["Data"].dt.day).astype(int)


#Definição do Dia da Semana
dic_dia_semana = {"Monday" : "Segunda-Feira",
                  "Tuesday" : "Terça-Feira",
                  "Wednesday" : "Quarta-Feira", 
                  "Thursday" : "Quinta-Feira",
                  "Friday" : "Sexta-Feira",
                  "Saturday" : "Sábado",
                  "Sunday" : "Domingo"}

df_final_tratado["Dia_Semana"] = df_final_tratado["Data"].dt.day_name()
df_final_tratado["Dia_Semana"] = df_final_tratado["Dia_Semana"].map(dic_dia_semana)

## 1 Validações Iniciais

##### 1.1 Recuperando as Linhas Mães

In [6]:
df_linhas_mae = pd.read_csv(local_destino + "\\" + 'df_linha_mae' + ".csv", encoding="latin-1", decimal=",")
linhas_maes = df_linhas_mae['Linha'].to_list()
df_linhas = None

In [7]:
outras_linhas = np.unique(df_final_tratado['Linha'])

## 2 Análise Exploratória

In [28]:
colunas_analisadas_linhas = ['Sub_Prefeitura', 'Zona', 'Ano', 'Mes', 'Linha']

colunas_analisadas_subpref = ['Sub_Prefeitura', 'Zona', 'Ano', 'Mes', 'Volume_Passageiros_BU_VT_DIN']
colunas_analisadas_zona = ['Zona', 'Ano', 'Mes', 'Volume_Passageiros_BU_VT_DIN']
colunas_analisadas_total = ['Ano', 'Mes', 'Volume_Passageiros_BU_VT_DIN']

In [20]:
df_final_total = df_final_tratado[df_final_tratado['Linha'].isin(linhas_maes)][colunas_analisadas_total].groupby(colunas_analisadas_total[0:2], as_index=False).sum('Volume_Passageiros_BU_VT_DIN')
df_final_zona = df_final_tratado[df_final_tratado['Linha'].isin(linhas_maes)][colunas_analisadas_zona].groupby(colunas_analisadas_zona[0:3], as_index=False).sum('Volume_Passageiros_BU_VT_DIN')
df_final_subpref = df_final_tratado[df_final_tratado['Linha'].isin(linhas_maes)][colunas_analisadas_subpref].groupby(colunas_analisadas_subpref[0:5], as_index=False).sum('Volume_Passageiros_BU_VT_DIN')

In [29]:
df_final_linhas = df_final_tratado[df_final_tratado['Linha'].isin(linhas_maes)][colunas_analisadas_linhas].groupby(colunas_analisadas_linhas[0:4], as_index=False).count()
df_final_linhas

,Sub_Prefeitura,Zona,Ano,Mes,Linha
0,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,1,62
1,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,2,56
2,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,3,62
3,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,4,60
4,ARICANDUVA-FORMOSA-CARRAO,LESTE,2014,5,62
...,...,...,...,...,...
1855,VILA MARIANA,SUL,2023,12,126
1856,VILA MARIANA,SUL,2024,1,100
1857,VILA MARIANA,SUL,2024,2,76
1858,VILA MARIANA,SUL,2024,3,106


In [22]:
df_final_total.to_csv(local_destino + "\\df_final_total.csv", index=False, decimal=",", encoding='latin-1', sep=";")
df_final_zona.to_csv(local_destino + "\\df_final_zona.csv", index=False, decimal=",", encoding='latin-1', sep=";")
df_final_subpref.to_csv(local_destino + "\\df_final_subpref.csv", index=False, decimal=",", encoding='latin-1', sep=";")

In [30]:
df_final_linhas.to_csv(local_destino + "\\df_final_linhas.csv", index=False, decimal=",", encoding='latin-1', sep=";")

In [ ]:
df_final_tratado[colunas_analisadas].describe()/1000

### 2.1 Distribuição dos Dados

In [ ]:
plt.rcParams['figure.figsize'] = [20, 8]
for linha in linhas_maes[0:4]:
    sns.histplot(df_final_tratado[df_final_tratado['Linha']==linha]['Volume_Passageiros_BU_VT_DIN'],
                kde=True)
    
    plt.title("Histograma dos Totais para a Linha: " + linha, font = fonte_1)
    plt.xlabel("Quantidade Total de Passageiros", font = fonte_2)
    plt.ylabel("Frequencia", font = fonte_2)
    plt.show()

### 2.2 Analise por Dia da Semana

In [ ]:
sns.boxplot(data = df_final_tratado[(df_final_tratado['Linha']==linha) & (np.logical_not(df_final_tratado['Dia_Semana'].isin(['Sábado', 'Domingo'])))  ][['Dia_Semana', 'Ano', 'Linha', 'Volume_Passageiros_BU_VT_DIN']],
y='Volume_Passageiros_BU_VT_DIN', x="Ano", hue='Dia_Semana')

In [ ]:
plt.rcParams['figure.figsize'] = [20, 8]

for linha in linhas_maes[0:3]:
    sns.boxplot(data = df_final_tratado[df_final_tratado['Linha']==linha][['Dia_Semana', 'Linha', 'Volume_Passageiros_BU_VT']],
                y='Volume_Passageiros_BU_VT', x="Dia_Semana", 
                palette="crest",
                order=dic_dia_semana.values())
    
    plt.title("Boxplot dos Totais para a Linha: " + linha, font = fonte_1)
    plt.xlabel("Dia da Semana", font = fonte_2)
    plt.ylabel("Contagem de Passageiros", font = fonte_2)
    plt.show()

#### 2.2.1 Analise por Dia da Semana (PÓS-PANDEMIA)

In [ ]:
#Crio um campo booleano no dataframe para permitir a comparação
df_final_tratado['Pos_Pandemia'] = np.where(df_final_tratado['Data']>=data_corte_pos_pandemia, "Pós-Pandemia", "Pré-Pandemia")

In [ ]:
for linha in linhas_maes[0:4]:
    sns.boxplot(data = df_final_tratado[(df_final_tratado['Linha']==linha) & (df_final_tratado['Data']>=data_corte_pre_pandemia)][['Dia_Semana', 'Pos_Pandemia', 'Linha', 'Volume_Passageiros_BU_VT_DIN']],
                y='Volume_Passageiros_BU_VT_DIN', x="Dia_Semana", hue='Pos_Pandemia',
                palette="flare",
                order=dic_dia_semana.values())
    
    plt.title("Boxplot do Volume Total de Passageiros para a Linha: " + linha, font = fonte_1)
    plt.xlabel("Dia da Semana", font = fonte_2)
    plt.ylabel("Quantidade Total de Passageiros", font = fonte_2)
    plt.legend(loc="upper right")
    plt.show()

In [ ]:
for linha in linhas_maes[0:4]:
    sns.histplot(data = df_final_tratado[(df_final_tratado['Linha']==linha) & (df_final_tratado['Data']>=data_corte_pre_pandemia)], 
                 x = 'Volume_Passageiros_BU_VT_DIN', hue='Pos_Pandemia', element="step",
                 kde=False, palette='flare')
    
    plt.title("Histograma dos Totais para a Linha: " + linha, font = fonte_1)
    plt.xlabel("Quantidade Total de Passageiros", font = fonte_2)
    plt.ylabel("Frequencia", font = fonte_2)
    plt.show()

### 2.3 Evolução Mensal da Quantidade de Passageiros

In [ ]:
plt.rcParams['figure.figsize'] = [20, 8]
for linha in linhas_maes[0:5]:
    grafico = sns.lineplot(data = df_final_tratado[(df_final_tratado['Linha']==linha)][['Ano', 'Mes', 'Volume_Passageiros_BU_VT_DIN']].groupby(['Ano', 'Mes'], as_index=False).sum('Volume_Passageiros_BU_VT_DIN'),
                           y = 'Volume_Passageiros_BU_VT_DIN',  x= 'Mes', hue = 'Ano',
                           palette='Paired', markers=True, linewidth = 2.5, marker='o')
    
    grafico.set_xticks(range(13)) 
    grafico.set_xticklabels(['', 'Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'], size=15)

    
    plt.title("Evolução Mensal do Passageiros para a Linha: " + linha, font = fonte_1)
    plt.xlabel("Mês", font = fonte_2)
    plt.ylabel("Quantidade Total de Passageiros", font = fonte_2)
    plt.legend()
    plt.show()

## 3 Decomposição das Séries Temporais

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf, pacf

In [ ]:
class Analise_Serie_Temporal():

    def analise_serie_temporal(self, df, linha, params={"Agregacao" : "Mensal",
                                                        "Modelo" : "additive",
                                                        "Testes" : ["decomposicao", "pacf", "acf"],
                                                        "Lag" : 12}):
        
        #Print
        print("DECOMPOSIÇÃO DA SÉRIE TEMPORAL PARA A LINHA:", linha)
        
        #Crio um df alternativa, cujo index será a Data
        df_linha = df[(df['Linha']==linha)][['Data', 'Volume_Passageiros_BU_VT_DIN']].groupby('Data', as_index=False).sum('Volume_Passageiros_BU_VT_DIN')
        df_linha = df_linha.set_index('Data').asfreq('D')
        
        
        #Agrego os dados de acordo com os parâmetros estabelecidos
        if params['Agregacao'] == "Mensal":
            df_linha = df_linha.resample('M').sum()
            periodo = 12
            print('Dados agragados a nível Mensal. Período = 12')
        
        elif params['Agregacao'] == "Semanal":
            df_linha = df_linha.resample('W-MON').sum()
            periodo = 7
            print('Dados agragados a nível Semanal. Período = 7')
            
        elif params['Agregacao'] == "Anual":
            df_linha = df_linha.resample('Y').sum()
            periodo = 1
            print('Dados agragados a nível Anual. Período = 1')
            
        else:
            print("Nenhum nível de agregação foi definido")
            print("Definindo automaticamente para Mensal...")
            df_linha = df_linha.resample('M').sum()
            
            
        if 'pacf' in params['Testes']:
            self.funcao_autocorrelacao(df_linha, params["Lag"])
            
        if 'decomposicao' in params['Testes']:
            dic_decomposicao = self.decomposicao(df_linha, periodo, params["Modelo"])
            return dic_decomposicao

    
    
    def decomposicao(self, df_linha, periodo, modelo='additive'):       
        plt.rcParams['figure.figsize'] = [20, 16]
        '''
        Para rodar a decomposição é preciso definir um modelo: Aditivo ou Multiplicativo.
        Se o modelo é ADITIVO, então Observed = Trend + Seasonal + Noise
        Se MULTIPLICATIVO, então Observed = Trend * Seasonal * Noise
        
        Assim sendo, usamos o modelo MULTIPLICATIVO quando a variação sazonal é DEPENDENTE da tendência, isto é,
        Tendência e Sazonalidade estão caminhando juntas e possuem uma correlação que requer a multiplicação
        
        O modelo ADITIVO é utilizado quando tendência e sazonalidade são independentes (são oriundas de processos geradores
        distintos), e assim são variáveis/fenômenos distintos que não possuem uma relação e NÃO PODEM SER MULTIPLICADOS.        
        '''
        
        decomposicao = sm.tsa.seasonal_decompose(df_linha.Volume_Passageiros_BU_VT_DIN.interpolate(), 
                                                 model=modelo,
                                                 period=periodo)
        sazonalidade = decomposicao.seasonal
        tendencia = decomposicao.trend
        residuo = decomposicao.resid

        dic_decomposicao = {"sazonalidade" : sazonalidade.values,
                            "tendencia" : tendencia.values, 
                            "residuo" : residuo.values}

        decomposicao.plot()        
        plt.show()
        
        #salvar_dados_decomposicao(decomposicao, sazonalidade, tendencia)
        
        return dic_decomposicao
    
    def funcao_autocorrelacao(self, df_linha, lag=12):
        plt.rcParams['figure.figsize'] = [10,6]
        sm.graphics.tsa.plot_pacf(df_linha.Volume_Passageiros_BU_VT_DIN.interpolate(),
                                  lags=lag)

In [ ]:
analise = Analise_Serie_Temporal()

In [ ]:
dic_dados_decompostos = {}

for linha in linhas_maes[0:2]:
    dic_dados_decompostos[linha] = analise.analise_serie_temporal(df_final_tratado, linha, params={"Agregacao" : "Mensal",
                                                                                                   "Modelo" : "multiplicative",
                                                                                                   "Testes" : ["decomposicao"],
                                                                                                   "Lag" : 12})

In [ ]:
#ANALISE DE AUTOCORRELAÇÃO PARCIAL
for linha in linhas_maes[0:10]:
    dic_dados_decompostos[linha] = analise.funcao_autocorrelacao(df_final_tratado, 12)